# **Imports**

## **numpy**

NumPy is an open source project aiming to enable numerical computing with Python. It was created in 2005, building on the early work of the Numeric and Numarray libraries. NumPy will always be 100% open source software, free for all to use and released under the liberal terms of the [modified BSD license](https://github.com/numpy/numpy/blob/main/LICENSE.txt).

NumPy is developed in the open on GitHub, through the consensus of the NumPy and wider scientific Python community. For more information on our governance approach, please see our [Governance Document](https://numpy.org/devdocs/dev/governance/index.html).

In [1]:
import numpy as np

## **bokeh**

Bokeh is a Python library for creating interactive visualizations for modern web browsers. It helps you build beautiful graphics, ranging from simple plots to complex dashboards with streaming datasets. With Bokeh, you can create JavaScript-powered visualizations without writing any JavaScript yourself.

In [2]:
from bokeh.io import output_notebook, show, export_png
from bokeh.colors import RGB
from bokeh.layouts  import row
from bokeh.plotting import figure

In [3]:
output_notebook()

## **tqdm**

tqdm derives from the Arabic word taqaddum (تقدّم) which can mean "progress".

Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!

In [4]:
from tqdm import tqdm

In [5]:
def green(s):
    return '\033[1;32m%s\033[m' % s

In [6]:
def red(s):
    return '\033[1;31m%s\033[m' % s

In [7]:
def log(*m):
    print(" ".join(map(str, m)))

# **Helper Functions**

## **Euclidian Distance**

In [8]:
def point_distance(marker: np.ndarray, x: np.ndarray) -> float:
    """ returns the euclian distance from point x to the marker point"""
    if marker is None or x is None:
        return np.Inf
    return np.linalg.norm(x-marker)

In [9]:
def get_k_nearest_points(marker: np.ndarray, k: int, points: np.ndarray, categories: np.ndarray) -> tuple:
    """ """
    distances = []
    for point in points:
        distance = point_distance(marker, point)
        distances.append(distance)
    distances = np.array(distances)

    sorted_index = np.argsort(distances)

    sorted_points = points[sorted_index]
    sorted_categories = categories[sorted_index]
    sorted_distances = distances[sorted_index]

    return sorted_points[:k, :], sorted_categories[:k], sorted_distances[:k] 

## **Binary Search**

In [10]:
def binary_search_recursive(value: float, sorted_list: list, point: np.ndarray, point_list: np.ndarray, left: int, right: int) -> tuple:
    """ """ 
    if left > right: 
        return False, left

    middle = left + (right - left) // 2

    if np.isclose(sorted_list[middle], value, rtol=1e-07) and np.isclose(point_list[middle], point, rtol=1e-07).all():
        return True, middle

    elif sorted_list[middle] > value:
        return binary_search_recursive(value, sorted_list, point, point_list, left, middle - 1)

    else:
        return binary_search_recursive(value, sorted_list, point, point_list, middle + 1, right)

In [11]:
def binary_search(value: float, sorted_list: list, point: np.ndarray, point_list: np.ndarray) -> tuple:
    """ """
    return binary_search_recursive(value, sorted_list, point, point_list, 0, len(sorted_list) - 1)

In [12]:
def get_y_dict(outputs: np.ndarray):
    y_dict, inverse = np.unique(outputs, return_inverse=True)

    y = []

    for i in range(outputs.size):
        out = outputs[i]
        out_index = inverse[i]

        out_array = np.zeros(y_dict.size)
        out_array[out_index] = 1

        y.append(out_array)

    return np.array(y), y_dict

## **Identical Points**

In [13]:
def is_point_in_array(point: np.ndarray, array: list) -> tuple:
    """ """
    for i in range(len(array)):
        if np.isclose(point, array[i]).all():
            return i
    return -1

In [14]:
def find_identical_points(x: np.ndarray, y: np.ndarray) -> tuple:
    uniques = []
    outputs = []
    weights = []

    outputs_dict = np.unique(y)

    for i in range(x.shape[0]):
        point = x[i]
        point_index = is_point_in_array(point, uniques)

        out = y[i]
        out_index = is_point_in_array(out, outputs_dict)

        out_array = np.zeros(outputs_dict.size)
        out_array[out_index] = 1

        if point_index == -1:
            uniques.append(point)
            weights.append(1)
            outputs.append(out_array)
        else:
            weights[point_index] += 1
            outputs[point_index] += out_array

    for i in range(len(outputs)):
        outputs[i] = outputs[i] / outputs[i].sum()
        
    return np.array(uniques), np.array(outputs), outputs_dict, np.array(weights)

In [15]:
def int_categories(array: np.ndarray) -> np.ndarray:
    unique = np.unique(array)
    result = []
    for item in array:
      i, = np.where(item == unique)
      result.append(i)
    return np.array(result).reshape(array.size)

## **Performance Measurement**

In [16]:
def compute_accuracy(predictions, y):
    """Computes the accuracy of predictions against the gold labels, y."""
    return np.mean(np.equal(predictions, y))

In [17]:
def compute_precision_recall(predictions, y):
    """Computes the precision and recall of predictions against the gold labels, y."""
    d = y.size

    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i in range(d):

        if y[i] == 1 and predictions[i] == 1:
            tp += 1

        elif y[i] == 1 and predictions[i] == 0:
            fn += 1

        elif y[i] == 0 and predictions[i] == 1:
            fp += 1
            
        else:
            tn += 1

    precision = tp / (tp + fp)

    recall = tp / (tp + fn)

    return precision, recall

# **Abstract Data Types**

## **Bounding Box**

In [18]:
class BB:
    """ """
    def __init__(self, d, lower, upper):
        self.d = d
        self.lower = lower
        self.upper = upper

    def set_lower_value(self, i, value):
        """ sets the value of the lower bound at position i """
        self.lower[i] = value;
        return self

    def set_upper_value(self, i, value):
        """ sets the value of the upper bound at position i """
        self.upper[i] = value;
        return self

    def copy(self):
        """ returns a copy of this bounding box """
        return BB(self.d, np.copy(self.lower), np.copy(self.upper))

    def contains(self, p):
        """ returns true if the bounding box contains the point p """
        return np.all(p >= self.lower) and np.all(p <= self.upper)

    def point_distance(self, p):
        """ returns the euclian distance from point p to the bounding box """
        if self.contains(p):
          return 0
        else:
          inner = 0
          for i in range(self.d):
            if p[i] < self.lower[i]:
              inner += (self.lower[i] - p[i])**2
            elif p[i] > self.upper[i]:
              inner += (self.upper[i] - p[i])**2
          return inner ** 0.5

## **k-d Tree**

In [19]:
class kd_tree:
    """ """
    def __init__(self, i, o):
        self.size, self.d = i.shape
        lower = i.min(axis=0)
        upper = i.max(axis=0)
        y, y_dict = get_y_dict(o)
        self.y_dict = y_dict
        self.n_classes = y_dict.size
        self.root = Node(0, i, y, self.n_classes, BB(self.d, lower, upper))

    def get_good_neighbor(self, point):
        """get the pseudo nearest neighbor"""
        return self.root.get_good_neighbor(point)

    def get_nearest_neighbor(self, point):
        """get the nearest neighbor"""
        best_point, best_y, best_distance = self.get_good_neighbor(point)
        return self.root.get_nearest_neighbor(point, best_point, best_y, best_distance)

    def get_k_good_neighbors(self, point, k):
        """get the k pseudo nearests neighbors"""
        return self.root.get_k_good_neighbors(point, k)

    def get_k_nearest_neighbors(self, point, k):
        """get the k nearests neighbors"""
        if k == 1:
            best_point, best_y, best_distance = self.get_nearest_neighbor(point)
            return np.array([best_point.tolist()]), np.array([best_y]), [best_distance], best_distance
        else: 
            best_points, best_y, best_distances = self.get_k_good_neighbors(point, k)
            greater_distance = best_distances[-1]
            return self.root.get_k_nearest_neighbors(point, k, best_points, best_y, best_distances, greater_distance)

    def predict(self, point, k, train=False, complete_return=False):
        """predict the label of one point"""
        best_points, best_y, best_distances, greater_distance = self.get_k_nearest_neighbors(point, k)

        if train: 
            best_y = best_y[1:]

        total_y = best_y.sum(axis=0)
        argmax = np.argmax(total_y)
        prediction = self.y_dict[argmax]

        if complete_return:
            return prediction, best_points, best_y, best_distances, greater_distance
        else:
            return prediction

    def plot_point(self, figure) -> None:
        """plot the point location"""
        if self.d == 2:
            self.root.plot_point(figure)
        else: 
            raise "dimension greater than two"

    def plot_line(self, figure) -> None:
        """plot the node decision lines"""
        if self.d == 2:
            self.root.plot_line(figure)
        else: 
            raise "dimension greater than two"

    def plot_rect(self, figure) -> None:
        """plot the node bb"""
        if self.d == 2:
            self.root.plot_rect(figure)
        else: 
            raise "dimension greater than two"

    def plot_prediction(self, marker, k, figure) -> None:
        """plot the knn prediction"""
        prediction = self.predict(marker, k)
        if prediction: 
            figure.scatter(marker[0], marker[1], size=9, fill_color="white", line_color="blue")
            figure.scatter(marker[0], marker[1], size=5, line_color="blue", marker="cross")
        else:
            figure.scatter(marker[0], marker[1], size=9, fill_color="white", line_color="red")
            figure.scatter(marker[0], marker[1], size=5, line_color="red", marker="dash")


### **Node**

In [20]:
class Node: 
    def __init__(self, i, x, y, n_classes, bb):
        self.size, self.d = x.shape

        self.column = i % self.d
        self.median = self.get_median(x)

        self.n_classes = n_classes
        self.bb = bb

        self.set_children(x, y)

    def get_median(self, x) -> float:
        """get the median of this node"""
        median = np.median(x[:, self.column])
        min = np.min(x[:, self.column])

        if median == min:
            return np.mean(x[:, self.column])
        else:   
            return median

    def get_filter(self, x) -> np.ndarray:
        """get the filter that corresponds to the
         hyperplane that separates the points"""
        return x[:, self.column] < self.median

    def get_child_matrices(self, x, y, filter) -> tuple:
        """get child x and y matrices"""
        return x[filter, :], y[filter, :]

    def get_child_bb(self, left) -> object:
        """get child bounding box"""
        bb_copy = self.bb.copy()

        if left: 
            bb_copy.set_upper_value(self.column, self.median) 
        else: 
            bb_copy.set_lower_value(self.column, self.median)

        return bb_copy

    def get_child(self, x, y, w, bb) -> object:
        """get child"""
        if x.shape[0] == 1:
            return Leaf(self.column + 1, x[0], y[0], w, bb)
        elif x.shape[0] == 0:
            return EmptyLeaf(self.d, self.n_classes)
        else: 
            return Node(self.column + 1, x, y, self.n_classes, bb)

    def set_children(self, x, y) -> tuple: 
        """set left and right children"""
        for i in range(self.d):

          l_filter = self.get_filter(x)

          l_x, l_y = self.get_child_matrices(x, y, l_filter)
          r_x, r_y = self.get_child_matrices(x, y, ~l_filter)

          # non empty children
          if l_y.size and r_y.size:
              w = 1
              break

          else:
              self.column = (self.column + 1) % self.d
              self.median = self.get_median(x)

        # all points are identical
        else:
              w = x.shape[0]
              x = x[0]
              x = x.reshape([1, self.d])

              y_total = y.sum(axis=0)
              y = y_total / y_total.sum()
              y = y.reshape([1, self.n_classes])

              l_filter = self.get_filter(x)

              l_x, l_y = self.get_child_matrices(x, y, l_filter)
              r_x, r_y = self.get_child_matrices(x, y, ~l_filter)

        l_bb, r_bb = self.get_child_bb(True), self.get_child_bb(False)

        self.l, self.r = self.get_child(l_x, l_y, w, l_bb), self.get_child(r_x, r_y, w, r_bb)

    def get_all_points(self) -> np.ndarray:
        """get all points below to this node"""
        l_points, l_y = self.l.get_all_points()
        r_points, r_y = self.r.get_all_points()
        return np.concatenate((l_points, r_points)), np.concatenate((l_y, r_y))

    def get_good_neighbor(self, point) -> tuple:
        """get pseudo nearest neighbor"""
        if point[self.column] < self.median and type(self.l) is not EmptyLeaf:
            return self.l.get_good_neighbor(point)
        else: 
            return self.r.get_good_neighbor(point)

    def get_k_good_neighbors(self, point, k) -> tuple:
        """get pseudo k nearests neighbors"""
        if point[self.column] < self.median:
            if self.l.size < k:
                all_points, all_y = self.get_all_points()
                return get_k_nearest_points(point, k, all_points, all_y)
            else:
                return self.l.get_k_good_neighbors(point, k)
        else: 
            if self.r.size < k:
                all_points, all_y = self.get_all_points()
                return get_k_nearest_points(point, k, all_points, all_y)
            else:
                return self.r.get_k_good_neighbors(point, k)

    def get_nearest_neighbor(self, point, best_point, best_y, best_distance) -> tuple:
        """get nearest neighbor"""
        if self.bb.point_distance(point) > best_distance:
            return best_point, best_y, best_distance

        best_point, best_y, best_distance = self.l.get_nearest_neighbor(point, best_point, best_y, best_distance)

        return self.r.get_nearest_neighbor(point, best_point, best_y, best_distance)
    
    def get_k_nearest_neighbors(self, point, k, best_points, best_y, best_distances, greater_distance): 
        """get k nearest neighbors"""
        if self.bb.point_distance(point) > greater_distance: 
            return best_points, best_y, best_distances, greater_distance

        best_points, best_y, best_distances, greater_distance = self.l.get_k_nearest_neighbors(point, k, best_points, best_y, best_distances, greater_distance)

        return self.r.get_k_nearest_neighbors(point, k, best_points, best_y, best_distances, greater_distance)

    def plot_point(self, figure) -> None:
        """plot the point location"""
        self.l.plot_point(figure)
        self.r.plot_point(figure)

    def plot_line(self, figure) -> None:
        """plot the node decision lines"""
        b = [self.median, self.median]
        if self.column == 0:
            figure.line(b, [self.bb.lower[1], self.bb.upper[1]], line_color="orange", line_width=1.5)
        else:
            figure.line([self.bb.lower[0], self.bb.upper[0]], b, line_color="orange", line_width=1.5)

        self.l.plot_line(figure)
        self.r.plot_line(figure)

    def plot_rect(self, figure) -> None:
        """plot the node bb"""
        self.l.plot_rect(figure)
        self.r.plot_rect(figure)

### **Leaf**

In [21]:
class Leaf:
    def __init__(self, i, point, y, w, bb):
        self.size, self.d = 1, point.size
        self.column = i % self.d
        self.bb = bb
        self.point = point
        self.y = y
        self.w = w

    def get_all_points(self) -> np.ndarray:
        """return this point w times"""
        return np.tile(self.point, (self.w, 1)), np.tile(self.y, (self.w, 1))

    def get_good_neighbor(self, point) -> tuple:
        """return this point"""
        return self.point, self.y, point_distance(point, self.point)

    def get_nearest_neighbor(self, point, best_point, best_y, best_distance) -> tuple:
        """return this point if it is closer to the best point"""
        this_distance = point_distance(self.point, point)
        if this_distance < best_distance:
            return self.point, self.y, this_distance
        else:
            return best_point, best_y, best_distance

    def get_k_nearest_neighbors(self, point, k, best_points, best_y, best_distances, greater_distance):
        """insert this point to the list if it is closer to the worst point""" 
        this_distance = point_distance(self.point, point)

        if this_distance < greater_distance:

          is_in_array, index = binary_search(this_distance, best_distances, self.point, best_points)

          if not is_in_array:

              for _ in range(self.w):
                  best_points = np.insert(best_points, index, self.point, axis=0)
                  best_y = np.insert(best_y, index, self.y, axis=0)
                  best_distances = np.insert(best_distances, index, this_distance)

              best_points = best_points[:k]
              best_y = best_y[:k]
              best_distances = best_distances[:k]

              greater_distance = best_distances[-1]

        return best_points, best_y, best_distances, greater_distance

    def plot_point(self, figure) -> None:
        """plot the point location"""
        red  = self.y[0] * 255 
        blue = self.y[1] * 255 

        color = RGB(red, 0, blue)

        figure.scatter(self.point[0], self.point[1], size=6 * self.w **0.5, fill_color=color, line_color=color)

    def plot_line(self, figure) -> None:
        """plot the node decision lines"""
        self.plot_point(figure)

    def plot_rect(self, figure) -> None:
        """plot the node bb"""
        sx = self.bb.lower[0]
        sy = self.bb.lower[1]

        w = self.bb.upper[0] - sx
        h = self.bb.upper[1] - sy

        figure.rect(sx + w/2, sy + h/2, w, h, color="orange", fill_alpha=0, line_width=1.5)

        self.plot_point(figure)

### **Empty Leaf**

In [22]:
class EmptyLeaf:
    def __init__(self, d, n_classes):
        self.size, self.d, self.n_classes = 0, d, n_classes

    def get_all_points(self) -> np.ndarray:
        """return empty point"""
        return np.zeros([0, self.d], dtype=float), np.zeros([0, self.n_classes], dtype=float)

    def get_nearest_neighbor(self, point, best_point, best_y, best_distance) -> tuple:
        """return the best point"""
        return best_point, best_y, best_distance

    def get_k_nearest_neighbors(self, point, k, best_points, best_y, best_distances, greater_distance) -> tuple:
        """return the k best points""" 
        return best_points, best_y, best_distances, greater_distance

    def plot_point(self, figure) -> None:
        """ do nothing"""
        return None

    def plot_line(self, figure) -> None:
        """ do nothing"""
        return None

    def plot_rect(self, figure) -> None:
        """ do nothing"""
        return None

# **Visualization**

In [23]:
n, d, c, k, marker = 39, 2, 2, 9, np.array([.5, .5])

In [24]:
def plot_kd_tree_visualization(tree, size=350) -> None:
    """plot the kd tree decision lines and bounding boxes"""
    f_ap = figure(plot_width=size, plot_height=size, match_aspect=True, title="Random Points")
    tree.plot_point(f_ap)

    f_dl = figure(plot_width=size, plot_height=size, match_aspect=True, title="Random Points - K-D Tree Decision Lines")
    tree.plot_line(f_dl)

    f_bb = figure(plot_width=size, plot_height=size, match_aspect=True, title="Random Points - K-D Tree Bounding Boxes")
    tree.plot_rect(f_bb)

    show(row(f_ap, f_dl, f_bb))

In [25]:
def plot_knn_visualization(tree, k, marker=np.array([0, 0]), size=350) -> None:
    """plot the nearest neighbor and the k nearest neighbors"""
    f_m = figure(plot_width=size, plot_height=size, match_aspect=True, title="Random Points - Marker")
    f_m.scatter(marker[0], marker[1], size=9, fill_color="white", line_color="black")
    tree.plot_point(f_m)

    nnx, nny, nn_radius = tree.get_nearest_neighbor(marker)
    f_nn = figure(plot_width=size, plot_height=size, match_aspect=True, title="Random Points - Nearest Neighbor")
    f_nn.scatter(marker[0], marker[1], radius=nn_radius, fill_color="orange", alpha=0.2)
    tree.plot_point(f_nn)
    f_nn.scatter(nnx[0], nnx[1], size=4, fill_color="white", line_color="white")
    tree.plot_prediction(marker, 1, f_nn)

    knnx, knny, _, knn_radius = tree.get_k_nearest_neighbors(marker, k)
    f_knn = figure(plot_width=size, plot_height=size, match_aspect=True, title="Random Points - K Nearest Neighbors")
    f_knn.scatter(marker[0], marker[1], radius=knn_radius, fill_color="orange", alpha=0.2)
    tree.plot_point(f_knn)
    f_knn.scatter(knnx[:, 0], knnx[:, 1], size=4, fill_color="white", line_color="white")
    tree.plot_prediction(marker, k, f_knn)

    show(row(f_m, f_nn, f_knn))

In [26]:
def show_kd_tree_distribution(i, c, n, marker):
    o = np.random.randint(c, size=n)

    tree = kd_tree(i, o)

    plot_kd_tree_visualization(tree)
    plot_knn_visualization(tree, k, marker)

## **Discrete Distributions** 

### **Uniform** 

In [27]:
i = np.random.randint(9, size=(n, d))
show_kd_tree_distribution(i, c, n, marker * 7)

### **Poisson**

In [28]:
i = np.random.poisson(3, size=(n, d))
show_kd_tree_distribution(i, c, n, marker * 7)

## **Continuous Distributions** 

### **Uniform** 

In [29]:
i = np.random.rand(n, d)
show_kd_tree_distribution(i, c, n, marker)

### **Triangular** 

In [30]:
i = np.random.triangular(0, 0.5, 1, size=(n, d))
show_kd_tree_distribution(i, c, n, marker)

### **Exponential** 

In [31]:
i = np.random.exponential(size=(n, d))
show_kd_tree_distribution(i, c, n, marker)

### **Normal** 

In [32]:
i = np.random.normal(0, 1, size=(n, d))
show_kd_tree_distribution(i, c, n, marker)

# **Tests**

In [33]:
from sklearn.neighbors import NearestNeighbors

In [34]:
n, d, k = 100, 10, 1

In [35]:
def test_knn(i, n, d, k, factor=0.5):
    """test the kd tree implementation"""
    o = np.ones(n) 

    tp_tree = kd_tree(i, o)

    sk_tree = NearestNeighbors(n_neighbors=k)
    sk_tree.fit(i)

    marker = np.ones(d) * factor

    tp_knn, _, _, _ = tp_tree.get_k_nearest_neighbors(marker, k)

    sk_knn_index = sk_tree.kneighbors(marker.reshape([1, d]), return_distance=False)
    sk_knn = i[sk_knn_index[0]]

    for i in range(tp_knn.shape[0]):
        if not np.isclose(tp_knn[i], sk_knn[i]).all(): 
            return False

    return True

In [36]:
def print_test_result(bool, n, d, k):
    if bool: 
        log(green("PASS:"), "\tn =", str(n), "\td =", str(d), " \tk =", k)
    else:
        log(red("FAIL"),  "\tn =", str(n), "\td =", str(d), " \tk =", k)

In [37]:
def print_multiple_tests(generator, n, d, k, factor=0.5):
    for a in range(4):
      for b in range(3):
        for c in range(2):
          new_n = n * 2**a
          new_d = d * 4**b
          new_k = k * 10**c
          i = generator(new_n, new_d)
          result = test_knn(i, new_n, new_d, new_k, factor)
          print_test_result(result, new_n, new_d, new_k)

## **Discrete Distributions** 

### **Uniform**

In [38]:
print_multiple_tests(lambda a, b : np.random.randint(1000, size=(a, b)), n, d, k, 500)

PASS: 	n = 100 	d = 10  	k = 1
PASS: 	n = 100 	d = 10  	k = 10
PASS: 	n = 100 	d = 40  	k = 1
PASS: 	n = 100 	d = 40  	k = 10
PASS: 	n = 100 	d = 160  	k = 1
PASS: 	n = 100 	d = 160  	k = 10
PASS: 	n = 200 	d = 10  	k = 1
PASS: 	n = 200 	d = 10  	k = 10
PASS: 	n = 200 	d = 40  	k = 1
PASS: 	n = 200 	d = 40  	k = 10
PASS: 	n = 200 	d = 160  	k = 1
PASS: 	n = 200 	d = 160  	k = 10
PASS: 	n = 400 	d = 10  	k = 1
PASS: 	n = 400 	d = 10  	k = 10
PASS: 	n = 400 	d = 40  	k = 1
PASS: 	n = 400 	d = 40  	k = 10
PASS: 	n = 400 	d = 160  	k = 1
PASS: 	n = 400 	d = 160  	k = 10
PASS: 	n = 800 	d = 10  	k = 1
PASS: 	n = 800 	d = 10  	k = 10
PASS: 	n = 800 	d = 40  	k = 1
PASS: 	n = 800 	d = 40  	k = 10
PASS: 	n = 800 	d = 160  	k = 1
PASS: 	n = 800 	d = 160  	k = 10


### **Poisson**

In [39]:
print_multiple_tests(lambda a, b : np.random.poisson(1000, size=(a, b)), n, d, k, 500)

PASS: 	n = 100 	d = 10  	k = 1
PASS: 	n = 100 	d = 10  	k = 10
PASS: 	n = 100 	d = 40  	k = 1
PASS: 	n = 100 	d = 40  	k = 10
PASS: 	n = 100 	d = 160  	k = 1
PASS: 	n = 100 	d = 160  	k = 10
PASS: 	n = 200 	d = 10  	k = 1
PASS: 	n = 200 	d = 10  	k = 10
PASS: 	n = 200 	d = 40  	k = 1
PASS: 	n = 200 	d = 40  	k = 10
PASS: 	n = 200 	d = 160  	k = 1
PASS: 	n = 200 	d = 160  	k = 10
PASS: 	n = 400 	d = 10  	k = 1
PASS: 	n = 400 	d = 10  	k = 10
PASS: 	n = 400 	d = 40  	k = 1
PASS: 	n = 400 	d = 40  	k = 10
PASS: 	n = 400 	d = 160  	k = 1
PASS: 	n = 400 	d = 160  	k = 10
PASS: 	n = 800 	d = 10  	k = 1
PASS: 	n = 800 	d = 10  	k = 10
PASS: 	n = 800 	d = 40  	k = 1
PASS: 	n = 800 	d = 40  	k = 10
PASS: 	n = 800 	d = 160  	k = 1
PASS: 	n = 800 	d = 160  	k = 10


## **Continuous Distributions** 

### **Uniform**

In [40]:
print_multiple_tests(lambda a, b : np.random.rand(a, b), n, d, k)

PASS: 	n = 100 	d = 10  	k = 1
PASS: 	n = 100 	d = 10  	k = 10
PASS: 	n = 100 	d = 40  	k = 1
PASS: 	n = 100 	d = 40  	k = 10
PASS: 	n = 100 	d = 160  	k = 1
PASS: 	n = 100 	d = 160  	k = 10
PASS: 	n = 200 	d = 10  	k = 1
PASS: 	n = 200 	d = 10  	k = 10
PASS: 	n = 200 	d = 40  	k = 1
PASS: 	n = 200 	d = 40  	k = 10
PASS: 	n = 200 	d = 160  	k = 1
PASS: 	n = 200 	d = 160  	k = 10
PASS: 	n = 400 	d = 10  	k = 1
PASS: 	n = 400 	d = 10  	k = 10
PASS: 	n = 400 	d = 40  	k = 1
PASS: 	n = 400 	d = 40  	k = 10
PASS: 	n = 400 	d = 160  	k = 1
PASS: 	n = 400 	d = 160  	k = 10
PASS: 	n = 800 	d = 10  	k = 1
PASS: 	n = 800 	d = 10  	k = 10
PASS: 	n = 800 	d = 40  	k = 1
PASS: 	n = 800 	d = 40  	k = 10
PASS: 	n = 800 	d = 160  	k = 1
PASS: 	n = 800 	d = 160  	k = 10


### **Triangular**

In [41]:
print_multiple_tests(lambda a, b : np.random.triangular(0, 0.5, 1, size=(a, b)), n, d, k)

PASS: 	n = 100 	d = 10  	k = 1
PASS: 	n = 100 	d = 10  	k = 10
PASS: 	n = 100 	d = 40  	k = 1
PASS: 	n = 100 	d = 40  	k = 10
PASS: 	n = 100 	d = 160  	k = 1
PASS: 	n = 100 	d = 160  	k = 10
PASS: 	n = 200 	d = 10  	k = 1
PASS: 	n = 200 	d = 10  	k = 10
PASS: 	n = 200 	d = 40  	k = 1
PASS: 	n = 200 	d = 40  	k = 10
PASS: 	n = 200 	d = 160  	k = 1
PASS: 	n = 200 	d = 160  	k = 10
PASS: 	n = 400 	d = 10  	k = 1
PASS: 	n = 400 	d = 10  	k = 10
PASS: 	n = 400 	d = 40  	k = 1
PASS: 	n = 400 	d = 40  	k = 10
PASS: 	n = 400 	d = 160  	k = 1
PASS: 	n = 400 	d = 160  	k = 10
PASS: 	n = 800 	d = 10  	k = 1
PASS: 	n = 800 	d = 10  	k = 10
PASS: 	n = 800 	d = 40  	k = 1
PASS: 	n = 800 	d = 40  	k = 10
PASS: 	n = 800 	d = 160  	k = 1
PASS: 	n = 800 	d = 160  	k = 10


### **Exponential**

In [42]:
print_multiple_tests(lambda a, b : np.random.exponential(size=(a, b)), n, d, k)

PASS: 	n = 100 	d = 10  	k = 1
PASS: 	n = 100 	d = 10  	k = 10
PASS: 	n = 100 	d = 40  	k = 1
PASS: 	n = 100 	d = 40  	k = 10
PASS: 	n = 100 	d = 160  	k = 1
PASS: 	n = 100 	d = 160  	k = 10
PASS: 	n = 200 	d = 10  	k = 1
PASS: 	n = 200 	d = 10  	k = 10
PASS: 	n = 200 	d = 40  	k = 1
PASS: 	n = 200 	d = 40  	k = 10
PASS: 	n = 200 	d = 160  	k = 1
PASS: 	n = 200 	d = 160  	k = 10
PASS: 	n = 400 	d = 10  	k = 1
PASS: 	n = 400 	d = 10  	k = 10
PASS: 	n = 400 	d = 40  	k = 1
PASS: 	n = 400 	d = 40  	k = 10
PASS: 	n = 400 	d = 160  	k = 1
PASS: 	n = 400 	d = 160  	k = 10
PASS: 	n = 800 	d = 10  	k = 1
PASS: 	n = 800 	d = 10  	k = 10
PASS: 	n = 800 	d = 40  	k = 1
PASS: 	n = 800 	d = 40  	k = 10
PASS: 	n = 800 	d = 160  	k = 1
PASS: 	n = 800 	d = 160  	k = 10


### **Normal**

In [43]:
print_multiple_tests(lambda a, b : np.random.normal(0, 1, size=(a, b)), n, d, k)

PASS: 	n = 100 	d = 10  	k = 1
PASS: 	n = 100 	d = 10  	k = 10
PASS: 	n = 100 	d = 40  	k = 1
PASS: 	n = 100 	d = 40  	k = 10
PASS: 	n = 100 	d = 160  	k = 1
PASS: 	n = 100 	d = 160  	k = 10
PASS: 	n = 200 	d = 10  	k = 1
PASS: 	n = 200 	d = 10  	k = 10
PASS: 	n = 200 	d = 40  	k = 1
PASS: 	n = 200 	d = 40  	k = 10
PASS: 	n = 200 	d = 160  	k = 1
PASS: 	n = 200 	d = 160  	k = 10
PASS: 	n = 400 	d = 10  	k = 1
PASS: 	n = 400 	d = 10  	k = 10
PASS: 	n = 400 	d = 40  	k = 1
PASS: 	n = 400 	d = 40  	k = 10
PASS: 	n = 400 	d = 160  	k = 1
PASS: 	n = 400 	d = 160  	k = 10
PASS: 	n = 800 	d = 10  	k = 1
PASS: 	n = 800 	d = 10  	k = 10
PASS: 	n = 800 	d = 40  	k = 1
PASS: 	n = 800 	d = 40  	k = 10
PASS: 	n = 800 	d = 160  	k = 1
PASS: 	n = 800 	d = 160  	k = 10


# **K-Nearest Neighbors**

In [44]:
class x_NN:
    """ """
    def __init__(self, x, y):
        self.n_classes = np.unique(y).size

        self.set_splitted_data(x, y)

        self.train_n, self.d = self.train_x.shape

        self.train_x_mean = self.train_x.mean(axis=0)
        self.train_x_std = self.train_x.std(axis=0)

        self.train_x = self.normalize(self.train_x)
        self.test_x = self.normalize(self.test_x)

        self.kd_tree = kd_tree(self.train_x, self.train_y)

    def set_splitted_data(self, x, y) -> None:
        """ """
        n = x.shape[0]
        limit = round(n * 0.7)

        permutation = np.random.permutation(n)
        
        x_unsorted = x[permutation]
        y_unsorted = y[permutation]

        self.train_x = x_unsorted[:limit, :]
        self.train_y = y_unsorted[:limit]

        self.test_x = x_unsorted[limit:, :]
        self.test_y = y_unsorted[limit:]

    def one_hot_encode(self, x) -> np.ndarray:
        """ """
        return 

    def normalize(self, array) -> np.ndarray:
        return (array - self.train_x_mean) / self.train_x_std[np.newaxis, :]

    def predict(self, k) -> np.ndarray:
        """ """
        results = []
        for point in tqdm(self.test_x): 
            point_prediction = self.kd_tree.predict(point, k)
            results.append(point_prediction)
        return np.array(results)

    def hyper_parameter_tuning(self) -> tuple:
        """ """
        upper = min(20, self.train_n // 10)

        predictions = np.zeros([upper - 1, self.test_x.shape[0]])

        for i in tqdm(range(self.test_x.shape[0])): 
            point = self.test_x[i]

            prediction, best_points, best_y, best_distances, greater_distance = self.kd_tree.predict(point, 1, complete_return=True)
            predictions[0][i] = prediction

            for k in range(2, upper):
                greater_distance = np.Inf
                best_points, best_y, best_distances, greater_distance = self.kd_tree.root.get_k_nearest_neighbors(point, k, best_points, best_y, best_distances, greater_distance)
                total_y = best_y.sum(axis=0)
                argmax = np.argmax(total_y)
                prediction = self.kd_tree.y_dict[argmax]
                predictions[k - 1][i] = prediction

        accuracy = []

        for i in range(upper - 1):
            predictions_i = predictions[i]
            accuracy.append(compute_accuracy(predictions_i, self.test_y))

        return (np.argmax(accuracy) + 1), accuracy

    def compute_accuracy(self, predictions):
        """ """
        return compute_accuracy(predictions, self.test_y)

    def compute_precision_recall(self, predictions):
        """ """
        return compute_precision_recall(predictions, self.test_y)

    def print_statistics(self, predictions) -> tuple:
        if self.n_classes == 2: 
            print('Test Accuracy:  {:.6f}'.format(self.compute_accuracy(predictions)))
            precision, recall = self.compute_precision_recall(predictions)
            print('Test Precision: {:.6f}'.format(precision))
            print('Test Recall:    {:.6f}'.format(recall))
        else: 
            print('Test Accuracy:  {:.6f}'.format(self.compute_accuracy(predictions)))

# **Datasets**

In [45]:
import pandas as pd

In [46]:
import requests
import io

In [47]:
ROOT = "https://raw.githubusercontent.com/magalhastudios1/k-nearest-neighbors/main/datasets/"

In [48]:
def import_dataset(path, skiprows=0) -> tuple: 
    """ """
    download = requests.get(path).content
    df = pd.read_csv(io.StringIO(download.decode('utf-8')), skiprows=skiprows, header=None)
    df = df.replace("?", np.nan)
    df = df.dropna()
    values = df.to_numpy()
    i = values[:, :-1]
    o = values[:, -1]
    return i.astype(np.float), int_categories(o)

In [49]:
def show_hyper_parameter_tuning(title, knn):
    """ """
    best_k, k_performance = knn.hyper_parameter_tuning()

    p = figure(plot_width=350, plot_height=350, title=title, x_axis_label="k", y_axis_label="accuracy")
    p.line(np.arange(1, len(k_performance) + 1), k_performance, line_width=2)

    predictions = knn.predict(best_k)
 
    accuracy = knn.compute_accuracy(predictions)

    p.scatter(best_k, accuracy, size=10, fill_color="white", line_color="red")
    show(p)

    return knn, predictions

In [50]:
def show_dataset(title, path, skiprows=0): 
    """ """
    i, o = import_dataset(path, skiprows)
    knn = x_NN(i, o)
    return show_hyper_parameter_tuning(title, knn)

## **Dermatology**

In [51]:
knn, predictions = show_dataset("Dermatology", ROOT + "/dermatology.dat", skiprows=39)

100%|██████████| 107/107 [00:01<00:00, 104.21it/s]


In [52]:
knn.print_statistics(predictions)

Test Accuracy:  0.971963


## **Zoo**

In [53]:
knn, predictions = show_dataset("Zoo", ROOT + "/zoo.dat", skiprows=21)

100%|██████████| 30/30 [00:00<00:00, 1053.37it/s]


In [54]:
knn.print_statistics(predictions)

Test Accuracy:  0.966667


## **Iris**

In [55]:
knn, predictions = show_dataset("Iris", ROOT + "/iris.dat", skiprows=9)

100%|██████████| 45/45 [00:00<00:00, 895.30it/s]


In [56]:
knn.print_statistics(predictions)

Test Accuracy:  0.955556


## **Mammographic**

In [57]:
knn, predictions = show_dataset("Mammographic", ROOT + "/mammographic.dat", skiprows=10)

100%|██████████| 249/249 [00:00<00:00, 645.95it/s]


In [58]:
knn.print_statistics(predictions)

Test Accuracy:  0.831325
Test Precision: 0.867769
Test Recall:    0.801527


## **Titanic**

In [59]:
knn, predictions = show_dataset("Titanic", ROOT + "/titanic.dat", skiprows=8)

100%|██████████| 660/660 [00:00<00:00, 3307.68it/s]


In [60]:
knn.print_statistics(predictions)

Test Accuracy:  0.781818
Test Precision: 0.907216
Test Recall:    0.394619


## **Marketing**

In [61]:
i, o = import_dataset(ROOT + "/marketing.dat", skiprows=18)
knn = x_NN(i, o)
predictions = knn.predict(5)

100%|██████████| 2063/2063 [00:53<00:00, 38.63it/s]


In [62]:
# k = 5
knn.print_statistics(predictions)

Test Accuracy:  0.282598


## **Winequality White**

In [63]:
i, o = import_dataset(ROOT + "/winequality-white.dat", skiprows=16)
knn = x_NN(i, o)
predictions = knn.predict(1)

100%|██████████| 1469/1469 [00:24<00:00, 59.57it/s]


In [64]:
# k = 1
knn.print_statistics(predictions)

Test Accuracy:  0.620150


## **Magic**

In [65]:
i, o = import_dataset(ROOT + "/magic.dat", skiprows=15)
knn = x_NN(i, o)
predictions = knn.predict(9)

100%|██████████| 5706/5706 [06:27<00:00, 14.71it/s]


In [66]:
# k = 9
knn.print_statistics(predictions)

Test Accuracy:  0.840694
Test Precision: 0.857143
Test Recall:    0.646013


## **Poker**

In [ ]:
i, o = import_dataset(ROOT + "/poker.dat", skiprows=15)
knn = x_NN(i, o)
predictions = knn.predict(1)

In [ ]:
# k = 1
knn.print_statistics(predictions)